In [7]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI

function = {
    "name": "create_quiz",
    "description": "function that takes a list of questions and answers and returns a quiz",
    "parameters": {
        "type": "object",
        "properties": {
            "questions": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "question": {
                            "type": "string",
                        },
                        "answers": {
                            "type": "array",
                            "items": {
                                "type" : "object",
                                "properties" : {
                                    "answer": {
                                        "type": "string",
                                    },
                                    "correct": {
                                        "type": "boolean",
                                    },
                                },
                                "required": ["answer", "correct"],
                            },
                        },
                    },
                    "required": ["question", "answers"],
                },
            },
        },
        "required": ["questions"],
    },
}

llm = ChatOpenAI(
    temperature=0.1,
).bind(
    function_call={
        "name": "create_quiz",
    },
    functions=[
        function,
    ],
)
prompt = PromptTemplate.from_template("Make a quiz about {city}")

chain = prompt | llm
response = chain.invoke({"city": "rome"})

response = response.additional_kwargs["function_call"]["arguments"]
response

'{"questions":[{"question":"What year was Rome founded?","answers":[{"answer":"753 BC","correct":true},{"answer":"476 AD","correct":false},{"answer":"1000 BC","correct":false}]},{"question":"Who was the first emperor of Rome?","answers":[{"answer":"Julius Caesar","correct":false},{"answer":"Augustus","correct":true},{"answer":"Nero","correct":false}]},{"question":"What famous structure in Rome was built by the ancient Romans?","answers":[{"answer":"Eiffel Tower","correct":false},{"answer":"Colosseum","correct":true},{"answer":"Big Ben","correct":false}]},{"question":"What language did the ancient Romans speak?","answers":[{"answer":"Latin","correct":true},{"answer":"Greek","correct":false},{"answer":"French","correct":false}]}]}'

In [8]:
import json
json.loads(response)

{'questions': [{'question': 'What year was Rome founded?',
   'answers': [{'answer': '753 BC', 'correct': True},
    {'answer': '476 AD', 'correct': False},
    {'answer': '1000 BC', 'correct': False}]},
  {'question': 'Who was the first emperor of Rome?',
   'answers': [{'answer': 'Julius Caesar', 'correct': False},
    {'answer': 'Augustus', 'correct': True},
    {'answer': 'Nero', 'correct': False}]},
  {'question': 'What famous structure in Rome was built by the ancient Romans?',
   'answers': [{'answer': 'Eiffel Tower', 'correct': False},
    {'answer': 'Colosseum', 'correct': True},
    {'answer': 'Big Ben', 'correct': False}]},
  {'question': 'What language did the ancient Romans speak?',
   'answers': [{'answer': 'Latin', 'correct': True},
    {'answer': 'Greek', 'correct': False},
    {'answer': 'French', 'correct': False}]}]}